In [ ]:
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils.quantization_config import BitsAndBytesConfig

In [ ]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

SYSTEM_PROMPT = """Respond in the following format:
<think>
You should reason between these tags.
</think>

Response goes here...

Always use <think> </think> tags even if they are not necessary.
"""


In [ ]:
def generate(model, tokenizer, prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        output = model.generate(input_ids, max_new_tokens=12288)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
def chat_with_ai(model, tokenizer):
    # Initialize conversation history
    conversation = [
        {"role": "system", "content": SYSTEM_PROMPT}
    ]

    print("Chat initialized. Type 'exit' to end the conversation.")

    while True:
        # Get user input
        user_message = input("You: ")

        if user_message.lower() == 'exit':
            break

        # Add user message to conversation history
        conversation.append({"role": "user", "content": user_message})

        # Apply chat template with proper tokenization
        inputs = tokenizer.apply_chat_template(
            conversation,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        # Move inputs to device and create attention mask
        input_ids = inputs.to(model.device)
        attention_mask = torch.ones_like(input_ids).to(model.device)
        prompt_length = input_ids.shape[1]

        # Generate response
        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=2048,  # Reduced from 12288 for more reasonable responses
            temperature=0.7,
            top_p=0.7,
            repetition_penalty=1.2,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

        # Extract just the new tokens
        new_tokens = output[0][prompt_length:]
        assistant_response = tokenizer.decode(new_tokens, skip_special_tokens=True)

        # Print the response
        print(f"AI: {assistant_response}")

        # Add assistant response to conversation history
        conversation.append({"role": "assistant", "content": assistant_response})

In [ ]:
chat_with_ai(model, tokenizer)